In [2]:
# !pip install -q openai 
import openai
import json
from tqdm import tqdm
from pathlib import Path

In [3]:
device = 'cuda:1'

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "roberta-base-openai-detector" # 0.65525
# model_name = "priyabrat/New_AI_or_Humantext_categorisation" 0.591

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.to(device)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [5]:
from human_ai.code.dataset import get_train_dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = get_train_dataset()
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [1]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

2023-03-12 22:35:30.014562: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 22:35:30.338934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 22:35:30.338989: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-12 22:35:31.531342: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
     model=model,
     args=training_args,
     train_dataset=tokenized_datasets['train'],
     eval_dataset=tokenized_datasets['test'],
     compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [10]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/homes/vladtom/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3600
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 675
  Number of trainable parameters = 124647170
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/homes/vladtom/

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.407204,0.850000
2,No log,0.668168,0.835000
3,0.257300,0.863219,0.855000


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 400
  Batch size = 16
/homes/vladtom/miniconda3/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 400
  Batch size = 16
/homes/vladtom/miniconda3/l

TrainOutput(global_step=675, training_loss=0.2076452269377532, metrics={'train_runtime': 358.3822, 'train_samples_per_second': 30.135, 'train_steps_per_second': 1.883, 'total_flos': 2841599397888000.0, 'train_loss': 0.2076452269377532, 'epoch': 3.0})

In [12]:
len(dataset['test'])

400

In [8]:
pred = []
labels = []

for sample in tqdm(train_data):
    # Transform input tokens 
    inputs = tokenizer(sample['text'], return_tensors="pt")
    # Model apply
    try:
        outputs = model(**inputs.to(device))
        pred.append(int(outputs.logits.argmax().cpu()))
    except:
        print(f"Error")
        pred.append(-1)
    labels.append(sample['label'])

NameError: name 'train_data' is not defined

In [23]:
from sklearn.metrics import accuracy_score

accuracy_score(labels, pred)

0.591

In [9]:
train_data[0]['text']

{'id': 0,
 'text': "Remains dating to the 5th century were found in tomb in Hwangnam-dong . Show a man 's bones on top of a woman 's who was buried with jewellery . Experts believe Silla Dynasty-era tomb was built for a noblewoman and her lover or bodyguard was sacrificed and buried on top of her . There 's a suggestion that the set-up may have been designed to show two people having sex - and the Silla were known for their explicit pottery .",
 'label': 1}

In [20]:
train_data[7]

{'id': 7,
 'text': 'Five Americans who were monitored for three weeks at an Omaha, Nebraska, hospital have been released. One of the five had a heart-related issue on Saturday and has been discharged. The others have already gone home.',
 'label': 0}